# Assignment 3.01: Satellite Data Download and Spectral Index Analysis in Google Earth Engine

## Learning Objectives

1. Select and define a study area of interest using GEE geometry tools
2. Download and process cloud-masked Sentinel-2 and Landsat imagery
3. Export properly scaled multi-band GeoTIFF files for downstream analysis
4. Calculate spectral indices (NDVI, SAVI, EVI, NDWI) in Google Earth Engine
5. Visualize and interpret spectral indices for your chosen landscape
6. Prepare analysis-ready datasets for use in Assignment 3.02

## Requirements

- Google Earth Engine account
- Access to Google Drive
- Basic understanding of JavaScript and GEE Code Editor
- Understanding of spectral indices and their applications

---

**Note:** You may utilize [r302_imagery_download](../resources/r302_imagery_download.ipynb) and [r301_imagery_indices](../resources/r301_imagery_indices.ipynb) as starting points.

**Important:** The satellite imagery you download and export in this assignment will be used directly in **Assignment 3.02 (a302)** for Python-based spectral index calculations. Make sure to save and organize your exported files carefully.

---

## Task 1: Select Your Study Area (2 points)

### Instructions:
Choose a study area of interest for spectral index analysis. **Important: Do NOT use Columbia, Missouri (COMO) since this was already demonstrated in the resource notebooks.** Pick a different location with diverse land cover types (vegetation, water, urban, agriculture).

**Suggested study areas within the USA:**

**Agricultural Regions:**
- Central Valley, California (crop diversity, irrigation patterns)
- Iowa Corn Belt (intensive agriculture)
- Imperial Valley, California (desert agriculture)

**Wetland and Water-Rich Areas:**
- Everglades, Florida (wetlands, water bodies)
- Chesapeake Bay, Maryland/Virginia (coastal wetlands)
- Louisiana Delta (river systems, wetlands)
- Lake Tahoe, California/Nevada (alpine lake)

**Forest and Vegetation Areas:**
- Olympic National Forest, Washington (dense temperate rainforest)
- Great Smoky Mountains, Tennessee/North Carolina (deciduous forest)
- Boundary Waters, Minnesota (boreal forest and lakes)

**Mixed Urban-Natural Areas:**
- Phoenix, Arizona (urban sprawl, desert vegetation)
- Atlanta, Georgia (urban forest interface)
- Seattle, Washington (urban, water, forest mix)

**Arid/Semi-Arid Regions:**
- Death Valley, California (extreme aridity, minimal vegetation)
- Painted Desert, Arizona (sparse vegetation patterns)
- Mojave Desert, California (desert vegetation gradients)

Your study area should be a **rectangular region approximately 10-20 km on each side** to ensure manageable file sizes and processing time.

### Steps:

1. **Open Google Earth Engine Code Editor** at [code.earthengine.google.com](https://code.earthengine.google.com)

2. **Define your study area** using rectangular geometry:
   ```javascript
   // Example: Rectangle around Phoenix, AZ
   // Adjust coordinates for your chosen area
   var rect = ee.Geometry.Rectangle([-112.20, 33.35, -111.90, 33.55], null, false);
   Map.centerObject(rect, 11);
   Map.addLayer(rect, {color: 'red'}, 'Study Area');
   ```

3. **Justify your selection** - In your report, explain:
   - Why you chose this area
   - What land cover types are present
   - What spectral indices would be most informative for this landscape

### What to submit:
- Screenshot showing your study area boundary on the GEE map
- Written justification (100-150 words) explaining your area selection

---

## Task 2: Download Sentinel-2 and Landsat Imagery (3 points)

### Instructions:
Following the workflow demonstrated in [r302_imagery_download](../resources/r302_imagery_download.ipynb), download cloud-masked, analysis-ready imagery for both Sentinel-2 and Landsat 8/9 platforms.

### Required Steps:

**A. Define temporal window:**
```javascript
// Choose a season appropriate for your area
// Growing season (Apr-Oct) for vegetation
// Or adjust based on your study objectives
var start = '2023-04-01';
var end = '2023-10-31';
```

**B. Sentinel-2 SR Harmonized Workflow:**
```javascript
// This is just an example, and you may need to revise it to get the correct results!
// Load and filter collection
var s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(rect)
  .filterDate(start, end)
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40));

// Cloud masking function using SCL band
function maskS2_SCL(image) {
  var scl = image.select('SCL');
  var mask = scl.neq(3)  // cloud shadow
    .and(scl.neq(8))      // medium clouds
    .and(scl.neq(9))      // high clouds
    .and(scl.neq(10))     // cirrus
    .and(scl.neq(11));    // snow/ice
  return image.updateMask(mask);
}

// Create composite and export
var s2Comp = s2.map(maskS2_SCL).median().clip(rect);
var s2Bands = ['B2','B3','B4','B8']; // Blue, Green, Red, NIR

Map.addLayer(s2Comp, {bands: ['B4','B3','B2'], min: 0, max: 3000, gamma: 1.1}, 'S2 True Color');

// Export to Google Drive
Export.image.toDrive({
  image: s2Comp.select(s2Bands).toFloat(),
  description: 'S2_YourAreaName_2023',  // Change to reflect your area
  folder: 'GEE_Exports',
  region: rect,
  scale: 10,
  crs: 'EPSG:4326',  // Or appropriate UTM zone for your area
  maxPixels: 1e13
});
```

**C. Landsat 8/9 SR Collection 2 Workflow:**
```javascript
// This is just an example, and you may need to revise it to get the correct results!
// Load both Landsat 8 and 9
var l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
  .filterBounds(rect).filterDate(start, end);
var l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
  .filterBounds(rect).filterDate(start, end);
var landsat = l8.merge(l9).filter(ee.Filter.lt('CLOUD_COVER', 60));

// Cloud masking with scaling
function maskL2_QA(image) {
  var qa = image.select('QA_PIXEL');
  var mask = qa.bitwiseAnd(1 << 3).eq(0)
    .and(qa.bitwiseAnd(1 << 4).eq(0))
    .and(qa.bitwiseAnd(1 << 5).eq(0))
    .and(qa.bitwiseAnd(1 << 7).eq(0));
  
  var optical = image.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'])
    .multiply(0.0000275).add(-0.2);
  return image.addBands(optical, null, true).updateMask(mask);
}

var lComp = landsat.map(maskL2_QA).median().clip(rect);
var lBands = ['SR_B2','SR_B3','SR_B4','SR_B5'];

Map.addLayer(lComp, {bands: ['SR_B4','SR_B3','SR_B2'], min: 0.02, max: 0.3, gamma: 1.1}, 'Landsat True Color');

Export.image.toDrive({
  image: lComp.select(lBands).toFloat(),
  description: 'L8L9_YourAreaName_2023',  // Change to reflect your area
  folder: 'GEE_Exports',
  region: rect,
  scale: 30,
  crs: 'EPSG:4326',  // Match Sentinel-2 CRS
  maxPixels: 1e13
});
```

### What to submit:
1. A single screenshot demonstrating the successful completion of the assignment. This image must clearly show two things simultaneously:

  - The Tasks panel with the successful export status for both the Sentinel-2 and Landsat GeoTIFF files (ensure your username is visible in the interface to confirm file ownership).

  - The map display showing the true color composites for both the Sentinel-2 and Landsat imagery.

2. The Downloaded GeoTIFF files saved locally. You will need these files for the follow-up assignment (a302).

---

## Task 3: Calculate Spectral Indices in GEE (3 points)

### Instructions:
Using the cloud-masked composites you created in Task 2, calculate NDVI, SAVI, EVI, and NDWI using Google Earth Engine. Follow the methods demonstrated in [r301_imagery_indices](../resources/r301_imagery_indices.ipynb).

### Code Template:

**A. Calculate NDVI (Normalized Difference Vegetation Index):**
```javascript
// This is just an example, and you may need to revise it to get the correct results!
// For Sentinel-2
var ndvi_s2 = s2Comp.normalizedDifference(['B8','B4']).rename('NDVI');
var ndviVis = {min: -0.2, max: 0.8, palette: ['blue','white','green']};
Map.addLayer(ndvi_s2, ndviVis, 'S2 NDVI');

// For Landsat
var ndvi_l = lComp.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI');
Map.addLayer(ndvi_l, ndviVis, 'Landsat NDVI');
```

**B. Calculate SAVI (Soil Adjusted Vegetation Index):**
```javascript
// Using expression method for Sentinel-2
var savi_s2 = s2Comp.expression(
  '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
    'NIR': s2Comp.select('B8'),
    'RED': s2Comp.select('B4'),
    'L': 0.5
  }).rename('SAVI');

var saviVis = {min: -0.2, max: 0.6, palette: ['brown','yellow','green']};
Map.addLayer(savi_s2, saviVis, 'S2 SAVI');
```

**C. Calculate EVI (Enhanced Vegetation Index):**
```javascript
// This is just an example, and you may need to revise it to get the correct results!
var evi_s2 = s2Comp.expression(
  '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
    'NIR': s2Comp.select('B8'),
    'RED': s2Comp.select('B4'),
    'BLUE': s2Comp.select('B2')
  }).rename('EVI');

var eviVis = {min: -0.2, max: 1.0, palette: ['blue','white','green']};
Map.addLayer(evi_s2, eviVis, 'S2 EVI');
```

**D. Calculate NDWI (Normalized Difference Water Index):**
```javascript
var ndwi_s2 = s2Comp.expression(
  '(GREEN - NIR) / (GREEN + NIR)', {
    'GREEN': s2Comp.select('B3'),
    'NIR': s2Comp.select('B8')
  }).rename('NDWI');

var ndwiVis = {min: -0.3, max: 0.3, palette: ['red','white','blue']};
Map.addLayer(ndwi_s2, ndwiVis, 'S2 NDWI');
```

### What to submit:
- Screenshots showing all four indices (NDVI, SAVI, EVI, NDWI) visualized for your study area
- Brief description (50-100 words per index) of spatial patterns you observe

---

## Assignment Report (10 points total)

### Instructions:
Write a comprehensive report (**maximum 4 pages including figures**) describing your work and findings. **Do not include code in the report.**

### Report Structure and Point Breakdown:

**1. Introduction (2 points)**
- Describe your study area location, extent, and characteristics
- Explain your rationale for selecting this area
- Describe the expected land cover types and why spectral indices are useful for this landscape
- State your objectives for the analysis

**2. Methods (2 points)**
- Describe the satellite datasets used (Sentinel-2 SR Harmonized and Landsat 8/9 Collection 2)
- Explain the temporal window chosen and why
- Describe cloud masking approaches for each sensor
- Explain the four spectral indices calculated (formulas, purpose, interpretation)
- **Do not include code**, but describe the workflow in prose

**3. Results (4 points)**
- **Include all required visualizations:**
  - Study area boundary map
  - True color composites (Sentinel-2 and Landsat)
  - All four spectral indices (NDVI, SAVI, EVI, NDWI) for at least one sensor
- **Describe patterns observed in each index:**
  - Where are the highest/lowest values?
  - What land cover types do these correspond to?
  - How do NDVI, SAVI, and EVI compare?
  - What does NDWI reveal about water distribution?
- **Compare Sentinel-2 vs. Landsat results** (if you calculated indices for both)
  - Differences in spatial detail
  - Consistency of index values
  - Any discrepancies and possible reasons

**4. Discussion and Conclusion (2 points)**
- Interpret your findings in the context of the landscape
- Discuss which indices were most informative for your study area and why
- Comment on sensor differences (Sentinel-2 vs. Landsat)
- Reflect on the utility of GEE for this type of analysis
- Mention how these exported datasets will be used in Assignment 3.02

### Requirements:
- **Maximum 4 pages** (including all figures and maps)
- Professional writing with proper spelling and grammar
- Clear, well-labeled figures with captions
- **Do not include any code in the report**
- Save as PDF for submission

---

## Submission Requirements

### Files to Submit

**I. Report (graded component):**
   - Submit your report as a PDF through Canvas
   - Name: `LastName_FirstName_a301_Report.pdf`
   - **Do not include code within the report**

**II. All Other Materials (via Git):**
   - **GEE JavaScript code** (save as `.txt` file; name: `LastName_FirstName_a301_GEE_Script.txt`)
   - **Screenshots** organized in a folder:
     - Study area boundary
     - Export tasks panel
     - True color composites (S2 and Landsat)
     - All spectral index visualizations
   - **Exported GeoTIFF files** (download from Google Drive and include):
     - `S2_YourAreaName_2023.tif`
     - `L8L9_YourAreaName_2023.tif`
     - **These files are REQUIRED for Assignment 3.02!**
   - Maintain clear file organization within the repository

**III. Completed AI-Usage form (via Git)**

---

## Grading Rubric (10 Points Total)

**Only the report will be graded. All other materials (code, screenshots, data files) are for your learning and will be used in a302, but will not be directly evaluated for a301.**

| Report Section | Points | Criteria |
|----------------|--------|----------|
| Introduction | 2 | Clear study area description, strong rationale, well-defined objectives |
| Methods | 2 | Comprehensive description of datasets, preprocessing, and indices; demonstrates understanding |
| Results | 4 | High-quality visualizations, detailed pattern descriptions, quantitative observations, sensor comparison |
| Discussion/Conclusion | 2 | Thoughtful interpretation, index comparison, reflection on GEE utility, connection to a302 |
| **Total** | **10** | **Professional writing, proper formatting, within 4-page limit** |

---

## Tips for Success

**Study Area Selection:**
- Choose areas with clear land cover diversity (vegetation gradients, water bodies, urban/rural mix)
- Avoid extremely cloudy regions (e.g., Pacific Northwest in winter)
- Consider phenological timing - growing season for vegetation indices
- Make sure your area has sufficient Sentinel-2 and Landsat coverage

**Technical Tips:**
- **Save your GEE script frequently** using the "Save" button
- Test your code on a small date range first before processing the full time series
- Use `print(collection.size())` to verify you have images after filtering
- If exports fail, try reducing `maxPixels` or increasing `scale`
- **Download and save your exported GeoTIFF files immediately** - you need them for a302!

**Cloud Masking Tips:**
- If you get too many black/no-data pixels, relax cloud filtering thresholds
- For very cloudy regions, expand your date range or use a drier season
- Check the true color composite before calculating indices to verify data quality

**Visualization Tips:**
- Adjust `min` and `max` values in visualization parameters based on your landscape
- Desert areas may have lower NDVI than forests - adjust accordingly
- Urban areas may show negative NDVI - this is normal
- Water bodies should show high (positive) NDWI values
- Use the Inspector tool to click on features and check actual index values

**Report Tips:**
- Use clear figure captions that explain what is shown
- Compare and contrast indices - don't just describe them individually
- Relate patterns to real landscape features (e.g., "high NDVI in forested areas")
- Be quantitative when possible (e.g., "NDVI ranges from -0.1 to 0.7")
- Proofread carefully before submitting

**File Organization for a302:**
- Create a dedicated folder for your exported imagery
- Use consistent, descriptive file names
- Keep track of which sensor, date range, and area each file represents
- **You MUST have these files for Assignment 3.02!**

---

## Help Resources

- **GEE Documentation:** https://developers.google.com/earth-engine/
- **Sentinel-2 SR Harmonized Info:** Search "S2_SR_HARMONIZED" in GEE Data Catalog
- **Landsat Collection 2 Info:** Search "LC08/C02/T1_L2" in GEE Data Catalog
- **Resource notebooks:** r301_imagery_indices.ipynb, r302_imagery_download.ipynb
- **Spectral Indices Database:** https://www.indexdatabase.de/

---

## Extension Challenges (Optional)

For additional practice and deeper analysis, consider:

1. **Additional Indices:** Calculate GNDVI, MSAVI2, or MNDWI
2. **Temporal Analysis:** Create monthly composites and analyze seasonal changes
3. **Threshold Classification:** Create binary masks (e.g., water where NDWI > 0.2)
4. **Histogram Analysis:** Use GEE's `ui.Chart.image.histogram()` to analyze index distributions
5. **Multi-Sensor Comparison:** Export indices from both sensors and compare quantitatively

---

*Remember: The primary deliverable for this assignment is your report, but the exported satellite imagery files are essential for Assignment 3.02. Make sure to download, organize, and save your GeoTIFF files carefully! This assignment teaches you the complete workflow from data acquisition through analysis - skills that are fundamental to remote sensing research and applications.*